In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"] = os.getenv("OPENAI_API_BASE")

整个框架分为这样三个部分。
- 数据源（Data Sources）：数据可以有很多种，包括 PDF 在内的非结构化的数据（Unstructured Data）、SQL 在内的结构化的数据（Structured Data），以及 Python、Java 之类的代码（Code）。在这个示例中，我们聚焦于对非结构化数据的处理。
- 大模型应用（Application，即 LLM App）：以大模型为逻辑引擎，生成我们所需要的回答。
- 用例（Use-Cases）：大模型生成的回答可以构建出 QA/ 聊天机器人等系统。

![](https://static001.geekbang.org/resource/image/73/87/73a46eecd42038961db9067e75de3387.jpg?wh=2509x799)

具体流程分为下面 5 步。
1. Loading：文档加载器把 Documents 加载为以 LangChain 能够读取的形式。
2. Splitting：文本分割器把 Documents 切分为指定大小的分割，我把它们称为“文档块”或者“文档片”。
3. Storage：将上一步中分割好的“文档块”以“嵌入”（Embedding）的形式存储到向量数据库（Vector DB）中，形成一个个的“嵌入片”。
4. Retrieval：应用程序从存储中检索分割后的文档（例如通过比较余弦相似度，找到与输入问题类似的嵌入片）。
5. Output：把问题和相似的嵌入片传递给语言模型（LLM），使用包含问题和检索到的分割的提示生成答案。

In [8]:
!pip install pypdf
!pip install docx2txt

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3973 sha256=21704021b1a0e94a66f79e3de5cc50eb87c48754f473a3c78efcb822cb925e31
  Stored in directory: c:\users\deliqiao\appdata\local\pip\cache\wheels\0f\0e\7a\3094a4ceefe657bff7e12dd9592a9d5b6487ef4338ace0afa6
Successfully built docx2txt


In [13]:
# loading
from langchain.document_loaders import TextLoader, PyPDFLoader, Docx2txtLoader

base_dir = './langchain_ref/02_文档QA系统/OneFlower'
documents = []

for file in os.listdir(base_dir):
    file_path = os.path.join(base_dir, file)
    if file.endswith('.txt'):
        loader = TextLoader(file_path)
    elif file.endswith('.pdf'):
        loader = PyPDFLoader(file_path)
    elif file.endswith('.docx'):
        loader = Docx2txtLoader(file_path)
    else:
        continue
    documents.extend(loader.load())

In [11]:
documents

[Document(page_content='易速鲜花集团 \n \n \n \n \n \n \n \n \n \n \n易速鲜花 服务中心  \n易速鲜花 股份有限公司  易\n速\n鲜\n花\n员\n工\n手\n册 ', metadata={'source': './langchain/02_文档QA系统/OneFlower\\易速鲜花员工手册.pdf', 'page': 0}),
 Document(page_content='易速鲜花集团 \n 1 董事长致辞  \n亲爱的同事：  \n您好！欢迎您加入 易速鲜花 旅游文化股份有限公司 ! \n我代表易速鲜花 对您的到来表示热烈的欢迎！并为公司拥有您\n这样优秀的员工而感到自豪和骄傲。 易速鲜花 将会因您的努力工作\n而稳步健康发展，将会因您的贡献而更加精彩。在此，我为您即将\n为易速鲜花 而付出的辛勤汗水表示诚挚的感谢！  \n易速鲜花 这个大家庭需要您、我、他每一位员工积极发扬“ 团\n结向上，完美无缺 ”的企业精神，以高度的主人翁责任感、使命感，\n与易速鲜花 同呼吸、共命 运，在各自的岗位上，勤奋敬业，尽职尽\n责，奋力拼搏。作为大家庭的一员，同仁之间应默契配合，相互接\n纳，取长补短，共同奋进，同舟共济。这样，我们就可以成为一个\n坚强的战斗堡垒， 在创 易速鲜花 发展的道路上披荆斩棘， 乘风破浪，\n无往不胜，所向披靡。最终以一流的服务、一流的管理、一流的信\n誉让我们的 易速鲜花 成为全国的一颗明珠。  \n希望各位以本手册为指南，共创 易速鲜花 美好明天 ! \n最后，诚挚地祝愿大家在公司工作愉快，前程似锦 ! \n \n \n                    签名： \n ', metadata={'source': './langchain/02_文档QA系统/OneFlower\\易速鲜花员工手册.pdf', 'page': 1}),
 Document(page_content='易速鲜花集团 \n 2 易速鲜花 晨会宣言  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n  \n今天 \n我开始新的生活  \n我要用全身心的爱迎接今天  \n我会用我全部的热情  \n关爱我的同事  \n热爱我们的公司  \n服务于我们的客

In [15]:
# splitting

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0)
chunked_documents = text_splitter.split_documents(documents)
chunked_documents

Created a chunk of size 442, which is longer than the specified 200
Created a chunk of size 377, which is longer than the specified 200


[Document(page_content='易速鲜花集团 \n \n \n \n \n \n \n \n \n \n \n易速鲜花 服务中心  \n易速鲜花 股份有限公司  易\n速\n鲜\n花\n员\n工\n手\n册', metadata={'source': './langchain_ref/02_文档QA系统/OneFlower\\易速鲜花员工手册.pdf', 'page': 0}),
 Document(page_content='易速鲜花集团 \n 1 董事长致辞  \n亲爱的同事：  \n您好！欢迎您加入 易速鲜花 旅游文化股份有限公司 ! \n我代表易速鲜花 对您的到来表示热烈的欢迎！并为公司拥有您\n这样优秀的员工而感到自豪和骄傲。 易速鲜花 将会因您的努力工作\n而稳步健康发展，将会因您的贡献而更加精彩。在此，我为您即将\n为易速鲜花 而付出的辛勤汗水表示诚挚的感谢！  \n易速鲜花 这个大家庭需要您、我、他每一位员工积极发扬“ 团\n结向上，完美无缺 ”的企业精神，以高度的主人翁责任感、使命感，\n与易速鲜花 同呼吸、共命 运，在各自的岗位上，勤奋敬业，尽职尽\n责，奋力拼搏。作为大家庭的一员，同仁之间应默契配合，相互接\n纳，取长补短，共同奋进，同舟共济。这样，我们就可以成为一个\n坚强的战斗堡垒， 在创 易速鲜花 发展的道路上披荆斩棘， 乘风破浪，\n无往不胜，所向披靡。最终以一流的服务、一流的管理、一流的信\n誉让我们的 易速鲜花 成为全国的一颗明珠。  \n希望各位以本手册为指南，共创 易速鲜花 美好明天 ! \n最后，诚挚地祝愿大家在公司工作愉快，前程似锦 ! \n \n \n                    签名：', metadata={'source': './langchain_ref/02_文档QA系统/OneFlower\\易速鲜花员工手册.pdf', 'page': 1}),
 Document(page_content='易速鲜花集团 \n 2 易速鲜花 晨会宣言  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n  \n今天 \n我开始新的生活  \n我要用全身心的爱迎接今天  \n我会用我全部的热情  \n关爱我的同事  \n热爱我们的公司  \n服务于我

### Embedding

词嵌入（Word Embedding）是自然语言处理和机器学习中的一个概念，它将文字或词语转换为一系列数字，通常是一个向量。简单地说，词嵌入就是一个为每个词分配的数字列表。这些数字不是随机的，而是捕获了这个词的含义和它在文本中的上下文。因此，语义上相似或相关的词在这个数字空间中会比较接近。 举个例子，通过某种词嵌入技术，我们可能会得到：“国王” -> [1.2, 0.5, 3.1, …]“皇帝” -> [1.3, 0.6, 2.9, …]“苹果” -> [0.9, -1.2, 0.3, …] 从这些向量中，我们可以看到“国王”和“皇帝”这两个词的向量在某种程度上是相似的，而与“苹果”这个词相比，它们的向量则相差很大，因为这两个概念在语义上是不同的。 词嵌入的优点是，它提供了一种将文本数据转化为计算机可以理解和处理的形式，同时保留了词语之间的语义关系。这在许多自然语言处理任务中都是非常有用的，比如文本分类、机器翻译和情感分析等。


向量数据库，也称为矢量数据库或者向量搜索引擎，是一种专门用于存储和搜索向量形式的数据的数据库。在众多的机器学习和人工智能应用中，尤其是自然语言处理和图像识别这类涉及大量非结构化数据的领域，将数据转化为高维度的向量是常见的处理方式。这些向量可能拥有数百甚至数千个维度，是对复杂的非结构化数据如文本、图像的一种数学表述，从而使这些数据能被机器理解和处理。然而，传统的关系型数据库在存储和查询如此高维度和复杂性的向量数据时，往往面临着效率和性能的问题。因此，向量数据库被设计出来以解决这一问题，它具备高效存储和处理高维向量数据的能力，从而更好地支持涉及非结构化数据处理的人工智能应用。

向量数据库有很多种，比如 Pinecone、Chroma 和 Qdrant，有些是收费的，有些则是开源的。

![](http://pic.dellyqiao.com:3011//202405021137644.png)


In [18]:
!pip install qdrant-client

  Using cached portalocker-2.8.2-py3-none-any.whl.metadata (8.5 kB)
   ---------------------------------------- 0.0/229.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/229.3 kB ? eta -:--:--
   ------ -------------------------------- 41.0/229.3 kB 495.5 kB/s eta 0:00:01
   ------------ -------------------------- 71.7/229.3 kB 660.6 kB/s eta 0:00:01
   ------------------------------ --------- 174.1/229.3 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 229.3/229.3 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ------- -------------------------------- 0.2/1.1 MB 6.1 MB/s eta 0:00:01
   ------------- -------------------------- 0.4/1.1 MB 4.6 MB/s eta 0:00:01
   ------------------------- -------------- 0.7/1.1 MB 5.5 MB/s eta 0:00:01
   ---------------------------------------  1.1/1.1 MB 6.1 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 5.7 MB/s eta 0:00:00
Using cached portalo

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googleapis-common-protos 1.63.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
opentelemetry-proto 1.24.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.26.1 which is incompatible.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 5.26.1 which is incompatible.


In [19]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings

qdrant = Qdrant.from_documents(
    documents=chunked_documents,
    embedding=OpenAIEmbeddings(),
    location=":memory:",
    collection_name="my_documents",
)

In [22]:
qdrant

### 相关信息的获取

当内部文档存储到向量数据库之后，我们需要根据问题和任务来提取最相关的信息。此时，信息提取的基本方式就是把问题也转换为向量，然后去和向量数据库中的各个向量进行比较，提取最接近的信息。

向量之间的比较通常基于向量的距离或者相似度。在高维空间中，常用的向量距离或相似度计算方法有欧氏距离和余弦相似度。

- 欧氏距离：这是最直接的距离度量方式，就像在二维平面上测量两点之间的直线距离那样。在高维空间中，两个向量的欧氏距离就是各个对应维度差的平方和的平方根。
- 余弦相似度：在很多情况下，我们更关心向量的方向而不是它的大小。例如在文本处理中，一个词的向量可能会因为文本长度的不同，而在大小上有很大的差距，但方向更能反映其语义。余弦相似度就是度量向量之间方向的相似性，它的值范围在 -1 到 1 之间，值越接近 1，表示两个向量的方向越相似。

关心数量等大小差异时用欧氏距离，关心文本等语义差异时用余弦相似度。

具体来说，欧氏距离度量的是绝对距离，它能很好地反映出向量的绝对差异。当我们关心数据的绝对大小，例如在物品推荐系统中，用户的购买量可能反映他们的偏好强度，此时可以考虑使用欧氏距离。同样，在数据集中各个向量的大小相似，且数据分布大致均匀时，使用欧氏距离也比较适合。

余弦相似度度量的是方向的相似性，它更关心的是两个向量的角度差异，而不是它们的大小差异。在处理文本数据或者其他高维稀疏数据的时候，余弦相似度特别有用。比如在信息检索和文本分类等任务中，文本数据往往被表示为高维的词向量，词向量的方向更能反映其语义相似性，此时可以使用余弦相似度。

在这里，我们正在处理的是文本数据，目标是建立一个问答系统，需要从语义上理解和比较问题可能的答案。因此，我建议使用余弦相似度作为度量标准。通过比较问题和答案向量在语义空间中的方向，可以找到与提出的问题最匹配的答案。

In [24]:
import logging
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains import RetrievalQA

logging.basicConfig(level=logging.INFO)
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
retriever = MultiQueryRetriever.from_llm(retriever=qdrant.as_retriever(), llm=llm)

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

### Generate answer